In [1]:
import sys
import os
import glob
import re 
import pandas as pd
import numpy as np
from lxml import etree
import matplotlib.pyplot as plt
import requests
import time
from collections import Counter


In [2]:
root = etree.parse('./../data/tei/de_gb14397.xml').getroot()

In [3]:
root

<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259fcaa72c8>

In [4]:
#!python -m spacy download zh_core_web_sm

In [5]:
import spacy as sp

c:\Users\calvotello\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\calvotello\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\calvotello\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\calvotello\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

In [6]:
nlp_de = sp.load("de_core_news_sm")
nlp_es = sp.load("es_core_news_sm")
nlp_en = sp.load("en_core_web_sm")
nlp_zh = sp.load("zh_core_web_sm")
nlp_fr = sp.load("fr_core_news_sm")

c:\Users\calvotello\Anaconda3\lib\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'de_core_news_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\calvotello\Anaconda3\lib\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'es_core_news_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\calvotello\Anaconda3\lib\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'fr_core_news_s

In [7]:
root.getroottree().write('./../data/tei/de_gb14397.xml', encoding="utf-8")


In [8]:
def annotate_text_and_save_as_tei(file_name, nlp_model, elements = ["ab"]):

    namespaces = {'tei':'http://www.tei-c.org/ns/1.0','xi':'http://www.w3.org/2001/XInclude'}

    root = etree.parse('./../data/tei/' + file_name + '.xml').getroot()
    xpath = " | ".join(['//tei:text//tei:'+element for element in elements])
    print(xpath)
    for element in root.xpath(xpath, namespaces = namespaces):
        text_element = "".join(element.xpath('.//text()', namespaces=namespaces, with_tail=True)).strip()
        element.text = ""

        annotated_paragraph = nlp_model(str(text_element))
        
        for token in annotated_paragraph:
            
            if token.pos_ == "PUNCT":
                token_tag = "pc"
            else:
                token_tag = "w"
            token_element = etree.SubElement(element, token_tag)
            token_element.text = token.text
            if file_name != "zh_Luxun_Kuangren":
                token_element.attrib['lemma'] = token.lemma_
            token_element.attrib['pos'] = token.pos_  
        
    root.getroottree().write('./../data/tei-annotated/' + file_name + '.xml', encoding="utf-8")

    
    return root

In [9]:
annotate_text_and_save_as_tei("de_gb14397", nlp_de, elements = ["ab"])

//tei:text//tei:ab


<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259890f2948>

In [14]:
annotate_text_and_save_as_tei("en_amdram1", nlp_en, elements = ["p","stage"])

//tei:text//tei:p | //tei:text//tei:stage


<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259a070fb08>

In [11]:
annotate_text_and_save_as_tei("es_ne0120", nlp_es, elements = ["p"])

//tei:text//tei:p


<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259960d9ac8>

In [12]:
annotate_text_and_save_as_tei("fr_Aarons_Attila", nlp_fr, elements = ["p"])

//tei:text//tei:p


<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259960e6988>

In [13]:
annotate_text_and_save_as_tei("zh_Luxun_Kuangren", nlp_zh, elements = ["p"])

//tei:text//tei:p


<Element {http://www.tei-c.org/ns/1.0}TEI at 0x259b6cf5dc8>